## Домашнее задание по теме урока: Логистическая регрессия. Log Loss.
**Постараюсь реализовать многоклассовую логрегрессию (частный случай которой - 2 класса**)

***Соответствующими комметариями буду указывать на реализации кода, которые соответствуют номерам ДЗ***

In [45]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [46]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2,  1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

In [47]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [48]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [49]:
X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  2.        ,  5.        ,  3.        ,
         0.        ,  5.        , 10.        ,  1.        ,  2.        ],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         1.        ,  3.        ,  3.        ,  1.        ,  2.        ]])

In [50]:
'''функция для преобразования вектора индексов классов в матрицу, содержащий
one-hot веткор для каждого экземпляра. В нашем случае это два экземпляра,
но данная реализация может быть использованап по множество экземпляров'''
def to_one_hot(y):
    n_classes = y.max() + 1
    m = len(y)
    Y_one_hot = np.zeros((m, n_classes))
    Y_one_hot[np.arange(m), y] = 1
    return Y_one_hot

In [51]:
y

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

In [52]:
Y_train_one_hot = to_one_hot(y)
Y_train_one_hot

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [53]:
#реализация функции softmax (как в scipy.special) (оно же, по сути, №3 домашки)
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exp_sums

In [54]:
# определим количество входов и выходов для обучения
n_inputs = X_st.T.shape[1] # == 4 (3 признака плюс смещение)
n_outputs = len(np.unique(y))   # == 2 класса
n_inputs, n_outputs

(4, 2)

#### 1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

*при реализации просто добавлю epsilon = 1e-7, это избавит как от нулей, так и от nan*

In [79]:
eta = 0.01
n_iterations = 5000
m = len(X_st)
epsilon = 1e-7

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_st.T.dot(Theta)#транспонирую X_st (здесь везде далее), чтобы соответствовать стандартным формулам
    Y_proba = softmax(logits)
    loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
    error = Y_proba - Y_train_one_hot
    if iteration % 500 == 0:
        print(iteration, loss)
    gradients = 1/m * X_st.dot(error)
    Theta = Theta - eta * gradients

0 3.7017819923396322
500 0.5274229425235799
1000 0.47240370090616135
1500 0.43704164597316264
2000 0.4096114369949804
2500 0.38765445447671254
3000 0.3698383448531706
3500 0.35519429215605125
4000 0.3429920436782424
4500 0.33268336530321674


In [80]:
Theta

array([[ 2.38073559, -2.016056  ],
       [ 1.11844771,  0.13294873],
       [ 0.14804978, -0.170559  ],
       [-2.04369481,  2.29935959]])

In [81]:
#ДЗ 3 и 4
logits = X_st.T.dot(Theta)
Y_proba = softmax(logits) # это есть номер 3 домашки
y_predict = np.argmax(Y_proba, axis=1) # это есть номер 4 домашки
Y_proba,y_predict

(array([[0.67417522, 0.32582478],
        [0.70235622, 0.29764378],
        [0.07826488, 0.92173512],
        [0.99129257, 0.00870743],
        [0.26488827, 0.73511173],
        [0.48469249, 0.51530751],
        [0.0335719 , 0.9664281 ],
        [0.8694506 , 0.1305494 ],
        [0.6669177 , 0.3330823 ],
        [0.09096634, 0.90903366]]),
 array([0, 0, 1, 0, 1, 1, 1, 0, 0, 1], dtype=int64))

In [82]:
y

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

In [83]:
# ДЗ 5
data = {'y_Actual':    y,
        'y_Predicted': y_predict
        }
df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
print (df)

   y_Actual  y_Predicted
0         0            0
1         0            0
2         1            1
3         0            0
4         1            1
5         0            1
6         1            1
7         0            0
8         1            0
9         1            1


In [84]:
# ДЗ 5
data = {'y_Actual':    y,
        'y_Predicted': y_predict
        }
df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
confusion_matrix = pd.crosstab(df['y_Actual'], df['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)

Predicted  0  1
Actual         
0          4  1
1          1  4


In [85]:
# ДЗ 5
accuracy = (confusion_matrix.iloc[1, 1]+confusion_matrix.iloc[0, 0])/(confusion_matrix.iloc[0, 0]+confusion_matrix.iloc[1, 0]+confusion_matrix.iloc[0, 1]+confusion_matrix.iloc[1, 1])
accuracy

0.8

In [86]:
# ДЗ 5
Precission = confusion_matrix.iloc[1, 1]/(confusion_matrix.iloc[1, 1] + confusion_matrix.iloc[0, 1])
Recall = confusion_matrix.iloc[1, 1]/(confusion_matrix.iloc[1, 1] + confusion_matrix.iloc[1, 0])
F = (2*Precission*Recall)/(Precission+Recall)
Precission, Recall, F

(0.8, 0.8, 0.8000000000000002)

## ДЗ 2 и 6

**Да модель могла переобучиться, потому что мы никак не ограничивали наши веса. Ниже я делаю реализацию регуляризации, а также прекращения градиентного спуска во избежание лишних иттераций (что соответствкет ДЗ 2 по сути)**

In [93]:
eta = 0.1 
n_iterations = 10001
m = len(X_st)
epsilon = 1e-7
alpha = 0.1  # гиперпараметр регуляризации
best_loss = np.infty

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_st.T.dot(Theta)
    Y_proba = softmax(logits)
    xentropy_loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
    l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
    loss = xentropy_loss + alpha * l2_loss
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_st.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]
    Theta = Theta - eta * gradients

    logits = X_st.T.dot(Theta)
    Y_proba = softmax(logits)
    xentropy_loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
    l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
    loss = xentropy_loss + alpha * l2_loss
    if iteration % 10 == 0:
        print(iteration, loss)
    if loss < best_loss:
        best_loss = loss
    else:
        print(iteration - 1, best_loss)
        print(iteration, loss, "early stopping!")
        break

0 2.2087298020618418
10 1.0807998509835701
20 0.9562263404698128
30 0.891962784827398
40 0.8440131668735087
50 0.803742739138587
60 0.7689964752745102
70 0.7382196134461858
80 0.7108012267336091
90 0.6864335814151389
100 0.664872015667269
110 0.6458910925797154
120 0.6292721887092488
130 0.6148014086399369
140 0.6022714181596752
150 0.5914841153897332
160 0.5822529149646493
170 0.5744042897626065
180 0.5677785728118444
190 0.5622301479132764
200 0.557627183581764
210 0.5538510514016974
220 0.5507955437939692
230 0.5483659790550671
240 0.5464782577766192
250 0.5450579155871145
260 0.5440392024156243
270 0.5433642075062665
280 0.5429820414570716
290 0.5428480809237464
291 0.5428467879995694
292 0.5428475463937765 early stopping!


In [94]:
Theta

array([[ 0.17679294, -1.06071302],
       [ 0.31553991, -0.34992096],
       [-0.24587814,  0.23122964],
       [-0.92169567,  0.97234298]])

In [95]:
logits = X_st.T.dot(Theta)
Y_proba = softmax(logits) # это есть номер 3 домашки
y_predict = np.argmax(Y_proba, axis=1) # это есть номер 4 домашки
Y_proba,y_predict

(array([[0.61687333, 0.38312667],
        [0.56942627, 0.43057373],
        [0.2692829 , 0.7307171 ],
        [0.95433784, 0.04566216],
        [0.26472392, 0.73527608],
        [0.38122693, 0.61877307],
        [0.16330058, 0.83669942],
        [0.76878507, 0.23121493],
        [0.62843175, 0.37156825],
        [0.22369742, 0.77630258]]),
 array([0, 0, 1, 0, 1, 1, 1, 0, 0, 1], dtype=int64))

In [96]:
data = {'y_Actual':    y,
        'y_Predicted': y_predict
        }
df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
confusion_matrix = pd.crosstab(df['y_Actual'], df['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)

Predicted  0  1
Actual         
0          4  1
1          1  4


In [97]:
accuracy = (confusion_matrix.iloc[1, 1]+confusion_matrix.iloc[0, 0])/(confusion_matrix.iloc[0, 0]+confusion_matrix.iloc[1, 0]+confusion_matrix.iloc[0, 1]+confusion_matrix.iloc[1, 1])
accuracy

0.8

In [98]:
Precission = confusion_matrix.iloc[1, 1]/(confusion_matrix.iloc[1, 1] + confusion_matrix.iloc[0, 1])
Recall = confusion_matrix.iloc[1, 1]/(confusion_matrix.iloc[1, 1] + confusion_matrix.iloc[1, 0])
F = (2*Precission*Recall)/(Precission+Recall)
Precission, Recall, F

(0.8, 0.8, 0.8000000000000002)

**результаты аналогичны (так получилось, нюанс этой мизернйо выборки) а модель работала быстрее и веса меньше изза регуляризации**